## Search trees

In [3]:
from util.load_lotr import lotr_chapters

chapters = lotr_chapters()

In [7]:
import re
from collections import defaultdict
from util.load_lotr import lotr_chapters

def preprocess_text(text):
    # Convert to lowercase and remove punctuation
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text.split()

def build_inverted_index(chapters):
    inverted_index = defaultdict(list)
    for chapter_idx, chapter in enumerate(chapters):
        words = preprocess_text(chapter)
        for word in set(words):  # Using set to avoid duplicate entries for the same chapter
            inverted_index[word].append(chapter_idx)
    return inverted_index

def search_chapters(inverted_index, query, chapters):
    query_words = preprocess_text(query)
    relevant_chapters = set()
    for word in query_words:
        if word in inverted_index:
            relevant_chapters.update(inverted_index[word])
    return [chapters[idx] for idx in sorted(relevant_chapters)]

# Build the inverted index
inverted_index = build_inverted_index(chapters)


In [ ]:
# Example search
query = "ring"
results = search_chapters(inverted_index, query, chapters)

# Print the results
for idx, chapter in enumerate(results):
    print(f"Chapter {idx + 1}:\n{chapter}\n")

## Binary / B-Tree

The tree structure you see is a Binary Search Tree (BST), which is a data structure that allows for efficient searching, insertion, and deletion operations. Let's break down why a BST is useful and how it is structured:

### Why Use a Binary Search Tree?

1. **Efficient Searching**: BSTs allow for efficient searching with an average time complexity of O(log n) for balanced trees. This means that, on average, you can find a term in the tree by examining only a fraction of the nodes, rather than all of them.

2. **Ordered Structure**: BSTs maintain elements in a sorted order, which can be useful for various operations like range queries and ordered traversals.

3. **Dynamic Data**: BSTs are well-suited for dynamic data, where elements are frequently inserted and deleted. They can maintain their structure and efficiency even as elements are added or removed.

### How is the BST Structured?

In a BST, each node has at most two children, referred to as the left child and the right child. The key property of a BST is that for any given node:

- All nodes in the left subtree have values less than the node's value.
- All nodes in the right subtree have values greater than the node's value.

This property ensures that the tree is always ordered in a specific way, which allows for efficient searching.

### Is the Tree Alphabetically Ordered?

Yes, in the context of your BST, the tree is alphabetically ordered. This means that when you traverse the tree in-order (left subtree, node, right subtree), you will visit the nodes in alphabetical order.

### Example of Searching in a BST

When you search for a term like "Aragorn" in the BST, the search process works as follows:

1. Start at the root node.
2. Compare the search term with the current node's term.
   - If the search term is less than the current node's term, move to the left child.
   - If the search term is greater than the current node's term, move to the right child.
   - If the search term is equal to the current node's term, you have found the term, and the associated data (e.g., chapter locations) can be retrieved.

### Example Traversal

Let's say you want to search for "Aragorn" in the BST. The traversal might look something like this:

1. Start at the root node ("Armies").
   - "Aragorn" is less than "Armies," so move to the left child.
2. Current node is "Arise."
   - "Aragorn" is less than "Arise," so move to the left child.
3. Current node is "Argonath."
   - "Aragorn" is less than "Argonath," so move to the left child.
4. Current node is "Archet."
   - "Aragorn" is less than "Archet," so move to the left child.
5. Current node is "Araw."
   - "Aragorn" is greater than "Araw," so move to the right child.
6. Current node is "Arathorn."
   - "Aragorn" is less than "Arathorn," so move to the left child.
7. Current node is "Aragorn."
   - "Aragorn" is equal to the current node's term, so the search is successful, and the chapter locations are retrieved.

### Benefits of Using a BST

- **Efficiency**: The BST structure allows for efficient searching, insertion, and deletion operations, which is particularly useful for large datasets.
- **Ordered Traversal**: The BST can be traversed in-order to retrieve elements in a sorted order, which can be useful for generating ordered lists or indexes.
- **Flexibility**: BSTs can handle dynamic data efficiently, making them suitable for applications where data is frequently updated.

In summary, using a BST for indexing and searching terms in your text provides an efficient and ordered way to manage and retrieve data, making it a suitable choice for many information retrieval tasks.

In [ ]:
import spacy
from util.load_lotr import lotr_chapters

nlp = spacy.load("en_core_web_sm")

chapters = lotr_chapters()

In [17]:
def extract_proper_nouns(text):
    doc = nlp(text)
    proper_nouns = [token.text for token in doc if token.pos_ == "PROPN"]
    return proper_nouns

class BSTNode:
    def __init__(self, term):
        self.term = term
        self.locations = []  # List to store locations where the term appears
        self.left = None
        self.right = None

class BST:
    def __init__(self):
        self.root = None

    def insert(self, term, location):
        if self.root is None:
            self.root = BSTNode(term)
            self.root.locations.append(location)
        else:
            self._insert_recursive(self.root, term, location)

    def _insert_recursive(self, node, term, location):
        if term < node.term:
            if node.left is None:
                node.left = BSTNode(term)
                node.left.locations.append(location)
            else:
                self._insert_recursive(node.left, term, location)
        elif term > node.term:
            if node.right is None:
                node.right = BSTNode(term)
                node.right.locations.append(location)
            else:
                self._insert_recursive(node.right, term, location)
        else:
            # Term already exists, just add the location
            node.locations.append(location)

    def search(self, term):
        return self._search_recursive(self.root, term)

    def _search_recursive(self, node, term):
        if node is None:
            return None
        if term == node.term:
            return node.locations
        elif term < node.term:
            return self._search_recursive(node.left, term)
        else:
            return self._search_recursive(node.right, term)

    def print_tree(self):
        if self.root is not None:
            self._print_recursive(self.root, 0)

    def _print_recursive(self, node, level):
        if node is not None:
            self._print_recursive(node.right, level + 1)
            print(' ' * 4 * level + '-> ' + node.term + ": " + str(node.locations))
            self._print_recursive(node.left, level + 1)


In [19]:

# Initialize the BST
bst = BST()

# Process each chapter
for chapter_idx, chapter in enumerate(chapters):
    proper_nouns = extract_proper_nouns(chapter)
    for noun in proper_nouns:
        bst.insert(noun, chapter_idx + 1)  # Using chapter index as location

In [20]:
# Example search for proper nouns
search_terms = ["Frodo", "Gandalf", "Aragorn", "Gollum", "Mordor"]
for term in search_terms:
    locations = bst.search(term)
    print(f"Locations where '{term}' appears:", locations)

Locations where 'Frodo' appears: [2, 6, 7, 11, 13, 14, 14, 15, 15, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 31, 31, 31, 31, 31, 31, 31, 31, 32, 32, 33, 33, 34, 34, 34, 34, 34, 34, 35, 35, 35, 35, 36, 36, 36, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 37, 37, 37, 37, 38, 42, 43, 43, 43, 43, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 49, 49, 49, 49, 49, 49, 49, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 62, 6

In [21]:
bst.print_tree()

                                -> Óin: [206, 226, 289]
                                                    -> Éowyn: [441, 441, 445, 446, 447, 447, 447, 447, 448, 448, 666, 669, 669, 669, 669, 669, 669, 673, 673, 673, 674, 674, 674, 676, 676, 676, 676, 676, 701, 701, 701, 701, 701, 702, 702, 702, 702, 702, 704, 704, 704, 705, 706, 709, 709, 709, 720, 720, 720, 722, 724, 724, 724, 726, 726, 732, 732, 732, 732, 732, 732, 732, 732, 732, 733, 735, 735, 739, 785, 785, 785, 785, 785, 785, 785, 785, 785, 785, 785, 785, 785, 785, 785, 792, 792, 792, 792, 792, 792, 792, 792, 793, 794, 802, 802, 802, 802, 802, 802, 802, 802, 802, 802]
                                                -> Éothain: [362, 363, 364]
                                            -> Éomund: [362, 362, 363, 363, 447, 448, 455, 470, 495, 701, 732]
                                        -> Éomer: [362, 362, 362, 362, 362, 362, 362, 362, 362, 362, 362, 362, 362, 362, 362, 363, 363, 363, 363, 363, 363, 363, 363, 363, 363, 363